In [2]:
import pandas as pd
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cbpro

public_client = cbpro.PublicClient()


In [3]:
class MinerMeta(type):
    def compile_historic(self, timestamp=False):
        start_date = datetime(2020, 1, 1)
        end_date = datetime.today()
        delta = timedelta(hours=300)
        df = pd.DataFrame()

        while start_date <= end_date:
            historic = public_client.get_product_historic_rates(
                f"{self.coin}-USD",
                granularity=3600,
                start=start_date,
                end=start_date + delta,
            )
            start_date += delta
            df = df.append(historic, ignore_index=True)
        df.columns = ["time", "low", "high", "open", "close", "volume"]
        date_time = pd.to_datetime(df["time"], unit="s")

        timestamp_s = date_time.map(pd.Timestamp.timestamp)
        day = 24 * 60 * 60
        year = (365.2425) * day
        df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
        df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
        df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
        df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))
        df.reset_index(drop=True, inplace=True)

        if timestamp is True:
            pass
        else:
            df["time"] = pd.to_datetime(df["time"], unit="s")
        # df["time"] = pd.to_datetime(df["time"], unit="s")
        # df["time"] = date_time.map(pd.Timestamp.timestamp)

        # data = np.array(df, dtype=np.float32)
        # ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        #     data=df,
        #     targets="close",
        #     sequence_length=10,
        #     sequence_stride=1,
        #     shuffle=False,
        #     batch_size=64,
        # )

        return df

    def plot_cols(self, df, cols=None):
        if cols == None:
            plot_features = df[list(df.columns)]
        else:
            plot_features = df[cols]

        index = df["time"]
        fig, axs = plt.subplots(len(plot_features))

        for x in len(plot_features):
            axs[x].plot(df["time"], plot_features[x])
        # fft = tf.signal.rfft(df[cols])

    def do_fft(self):
        df = self.compile_historic()
        today = datetime.today()

        fft = tf.signal.rfft(df["close"])
        f_per_dataset = np.arange(0, len(fft))

        n_samples_h = len(df["close"])
        hours_per_year = 24 * 365.2524
        years_per_dataset = n_samples_h / (hours_per_year)

        f_per_year = f_per_dataset / years_per_dataset
        plt.step(f_per_year, np.abs(fft))
        plt.xscale("log")
        plt.ylim(0, 400000)
        plt.xlim([0.1, max(plt.xlim())])
        plt.xticks([1, 365.2524], labels=["1/Year", "1/day"])
        _ = plt.xlabel("Frequency (log scale)")

        return plt.show()

    # def make_dataset(data):
    #     data = np.array(data, dtype=np.float32)
    #     ds = tf.keras.preprocessing.timeseries_dataset_from_array(
    #         data=data,
    #         targets="close",
    #         sequence_length=10,
    #         sequence_stride=1,
    #         shuffle=False,
    #         batch_size=64,
    #     )

    #     return ds

    def __call__(self, *args, **kwargs):

        cls = type.__call__(self, *args)

        setattr(cls, "compile_historic", self.compile_historic)
        setattr(cls, "plot_cols", self.plot_cols)
        setattr(cls, "do_fft", self.do_fft)
        # setattr(cls, "make_dataset", self.compile_historic)

        # for key, value in historic.items():
        #     setattr(cls, "hist_" + key, value)
        # for key, value in ticker.items():
        #     setattr(cls, "tick_" + key, value)

        return cls


class eth(metaclass=MinerMeta):
    coin = "eth"


class btc(metaclass=MinerMeta):
    coin = "btc"


class FeedBack(tf.keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = tf.keras.layers

    def warmup(self, inputs):
        # inputs.shape => (batch, time, features)
        # x.shape => (batch, lstm_units)
        x, *state = self.lstm_rnn(inputs)

        # predictions.shape => (batch, features)
        prediction = self.dense(x)
        return prediction, state

    def call(self, inputs, training=None):
        # Use a TensorArray to capture dynamically unrolled outputs.
        predictions = []
        # Initialize the LSTM state.
        prediction, state = self.warmup(inputs)

        # Insert the first prediction.
        predictions.append(prediction)

        # Run the rest of the prediction steps.
        for n in range(1, self.out_steps):
            # Use the last prediction as input.
            x = prediction
            # Execute one lstm step.
            x, state = self.lstm_cell(x, states=state, training=training)
            # Convert the lstm output to a prediction.
            prediction = self.dense(x)
            # Add the prediction to the output.
            predictions.append(prediction)

        # predictions.shape => (time, batch, features)
        predictions = tf.stack(predictions)
        # predictions.shape => (batch, time, features)
        predictions = tf.transpose(predictions, [1, 0, 2])
        return predictions


In [4]:
from pathlib import Path

file = Path.cwd() / "ethdata.json"


def plot_cols(self, df, cols=None):
    if cols == None:
        plot_features = df[list(df.columns)]
    else:
        plot_features = df[cols]

    index = df["time"]
    plots = len(plot_features)
    fig, axs = plt.subplots(plots)

    for x in plots:
        print(x)
        # axs[x].plot(index, plot_features[x])
    # fft = tf.signal.rfft(df[cols])


df = eth.compile_historic()

df.to_json(df)

TypeError: argument of type 'method' is not iterable

In [ ]:
class WindowGenerator:
    def __init__(
        self,
        input_width,
        label_width,
        shift,
        train_df=train_df,
        val_df=val_df,
        test_df=test_df,
        label_columns=None,
    ):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {
                name: i for i, name in enumerate(label_columns)
            }
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return "\n".join(
            [
                f"Total window size: {self.total_window_size}",
                f"Input indices: {self.input_indices}",
                f"Label indices: {self.label_indices}",
                f"Label column name(s): {self.label_columns}",
            ]
        )
